In [1]:
import json
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import nltk

In [2]:
with open("reviews_Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [3]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

{"reviewerID": "AAXUNK0W2DZG5", "asin": "0060520841", "reviewerName": "Amazon Customer \"leneker\"", "helpful": [5, 10], "reviewText": "1996 Bernard Goldberg wrote an editorial for the Wall Street Journal that said there was an obvious bias on the part of network new shows  for the liberal point of view.  he then illustrated this with an example that he dissected in-depth.  The reaction to this observation was the ruination of her career, and the beginning of his status as a pariah among most newsmen.  This book is used to add more ammo to the controversy.That the journalists who so eagerly pry into other peoples lives and business should be reluctant to be examined is hardly surprising.  Almost no one really wants to have his life or business dissected by Mike Wallace not even Dan Rather.  Some facts in this book are really potent such as the survey results which show how the average journalist and the average American are often vastly at odds.  Other chapters highlight different stor

In [4]:
review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)

In [5]:
import random
random_data = random.sample(review_data,len(review_data))

In [6]:
reviewText = []
for i in range(len(random_data)):
    reviewText.append(random_data[i]["reviewText"])

In [ ]:
def pos_tagging(document):
    result = nltk.pos_tag( nltk.word_tokenize(document))    
    last_result = ['/'.join(t) for t in result]
    return last_result

In [ ]:
tag = [pos_tagging(i) for i in reviewText]
tagging_VB_RB_JJ = []
for i in range(len(reviewText[:1000])):
    tagging = []
    if i % 100 == 0 :
        print(i)
    for j in tag[i]:
        if j.split("/")[1] == "JJ" or j.split("/")[1] == "JJR" or j.split("/")[1] == "JJS" or j.split("/")[1] == "RB" or j.split("/")[1] == "RBR" or j.split("/")[1] == "RBS" or j.split("/")[1] == "VB" or j.split("/")[1] == "VBD" or j.split("/")[1] == "VBG" or j.split("/")[1] == "VBN" or j.split("/")[1] == "VBP" or j.split("/")[1] == "VBZ":
            tagging.append(j.split("/")[0])
    join_each_tag = " ".join(tagging)
    tagging_VB_RB_JJ.append(join_each_tag)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_df = 0.3, min_df = 2, lowercase=True, stop_words = 'english')
D = vectorizer.fit_transform(tagging_VB_RB_JJ)

In [ ]:
D.shape

In [ ]:
y_data = []
int_y_data = []
for i in range(len(random_data)):
    y_data.append(random_data[i]["overall"])
y_data=np.array(y_data)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(D, y_data, test_size=0.3, random_state=100)

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

### LogisticRegression

In [38]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression()

logreg.fit(x_train, y_train)
logreg.predict(x_test) == y_test

array([ True, False,  True, ...,  True,  True,  True], dtype=bool)

In [40]:
from sklearn.metrics import accuracy_score
accuracy_score(logreg.predict(x_test), y_test)

0.61156817856631851

### Multiclass SVM

In [16]:
from sklearn import datasets
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

result = OneVsRestClassifier(LinearSVC(random_state=0)).fit(x_train, y_train)
result.predict(x_test)

array([ 5.,  5.,  5., ...,  2.,  5.,  1.])

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(result.predict(x_test),y_test)

0.59350407783660042